In [1]:
from transformers import pipeline
from transformers import AutoTokenizer

from numpy import dot
from numpy.linalg import norm

import re

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We first define some utility functions

In [2]:
# BERT maskierten satz geben
def bert_predict(sentence):
    model = pipeline('fill-mask', model='bert-base-uncased')
    pred = model(sentence)
    # bert wahrscheinlichkeiten neu verteilen
    
    return pred

# cosine similarity
def get_cosine_sim(A: list[float], B: list[float]) -> float:
    return dot(A, B)/(norm(A)*norm(B))

# takes in a sentence and returns an array with sentences containing each one [UNK] token from left to right.
# [MASK] won't be mapped to [UNK]
def get_sub_sentences(sentence: str) -> list[str]:
    sub_sentences = list()
    tokens = sentence[:-1].split(' ')
    
    for i in range(len(tokens)):
        s = ""
        for j in range(len(tokens)):
            if i == j:
                if tokens[j] == '[MASK]':
                    s += '[MASK] '
                else:
                    s += '[UNK] '
            else:
                s += tokens[j] + ' '

        sub_sentences.append(s[:-1] + '.')

    return sub_sentences

Yess... I am repeating a lot of code here!!
Python just wouldn't let me import it from other modules! Weird!

In [3]:
from nltk.stem import WordNetLemmatizer
import numpy as np
import math

IA = {'use': 76, 'apply': 16, 'take': 13, 'be': 10, 'attach': 9, 'create': 9, 'wield': 9, 'kill': 8, 'power': 6, 'play': 5, 'clean': 5, 'build': 5, 'form': 5, 'manipulate': 5, 'drive': 5, 'carry': 4, 'work': 4, 'fix': 4, 'mark': 4, 'give': 4, 'turn': 4, 'adjust': 4, 'show': 4, 'perform': 4, 'scale': 4, 'find': 3, 'stir': 3, 'choose': 3, 'acces': 3, 'determine': 3, 'treat': 3, 'tear': 3, 'fly': 3, 'threaten': 3, 'stop': 3, 'strike': 3, 'proces': 3, 'have': 3, 'start': 3, 'br': 3, 'move': 2, 'implement': 2, 'break': 2, 'calibrate': 2, 'approach': 2, 'resolve': 2, 'assist': 2, 'write': 2, 'cover': 2, 'identify': 2, 'study': 2, 'make': 2, 'touch': 2, 'help': 2, 'bake': 2, 'offer': 2, 'improve': 2, 'carve': 2, 'employ': 2, 'receive': 2, 'buy': 2, 'come': 2, 'win': 2, 'relax': 2, 'wear': 2, 'eat': 2, 'push': 2, 'put': 2, 'examine': 2, 'need': 2, 'remove': 2, 'deface': 2, 'obtain': 2, 'batter': 2, 'observe': 2, 'hold': 2, 'sweep': 2, 'operate': 2, 'travel': 2, 'get': 2, 'saw': 2, 'establish': 2, 'swap': 1, 'propose': 1, 'sew': 1, 'peel': 1, 'benefit': 1, 'plow': 1, 'catch': 1, 'drink': 1, 'filter': 1, 'locate': 1, 'asses': 1, 'mix': 1, 'date': 1, 'coat': 1, 'regain': 1, 'pres': 1, 'watch': 1, 'conduct': 1, 'machine': 1, 'light': 1, 'diagnose': 1, 'search': 1, 'complete': 1, 'reach': 1, 'sever': 1, 'encrypt': 1, 'draw': 1, 'Ad': 1, 'finish': 1, 'illuminate': 1, 'connect': 1, 'decrease': 1, 'crate': 1, 'block': 1, 'call': 1, 'time-travel': 1, 'sequence': 1, 'hail': 1, 'hide': 1, 'release': 1, 'produce': 1, 'assemble': 1, 'demonstrate': 1, 'hear': 1, 'associate': 1, 'try': 1, 'belong': 1, 'cool': 1, 'place': 1, 'estimate': 1, 'toil': 1, 'cure': 1, 'injure': 1, 'liberate': 1, 'distinguish': 1, 'ease': 1, 'facilitate': 1, 'cultivate': 1, 'blast': 1, 'begin': 1, 'reclaim': 1, 'sculpt': 1, 'tap': 1, 'staple': 1, 'teach': 1, 'achieve': 1, 'pull': 1, 'transform': 1, 'attend': 1, 'pas': 1, 'clean.': 1, 'install': 1, 'provide': 1, 'till': 1, 'screen': 1, 'conquer': 1, 'inject': 1, 'water': 1, 'set': 1, 'furnish': 1, 'purchase': 1, 'aid': 1, 'repair': 1, 'cope': 1, 'split': 1, 'fit': 1, 'deliver': 1, 'link': 1, 'in': 1, 'pick': 1, 'compare': 1, 'feather': 1, 'fry': 1, 'generate': 1, 'admit': 1, 'design': 1, 'punch': 1, 'learn': 1, 'cook': 1, 'stay': 1, 'attempt': 1, 'stand': 1, 'swim': 1, 'cut': 1, 'rebuild': 1, 'label': 1, 'capture': 1, 'execute': 1, 'interpret': 1, 'regulate': 1, 'solve': 1, 'construct': 1, 'queue': 1, 'destroy': 1, 'chase': 1, 'protect': 1, 'leave': 1, 'kick': 1, 'prefer': 1, 'join': 1, 'combine': 1, 'hit': 1, 'feature': 1, 'enable': 1, 'accomplish': 1, 'prepare': 1, 'utilize': 1, 'sell': 1, 'squeeze': 1, 'crush': 1, 'pound': 1, 'insert': 1, 'keep': 1, 'fight': 1, 'raise': 1, 'grab': 1, 'permit': 1, 'down': 1, 'throw': 1, 'buck': 1}

CE = {'cause': 423, 'be': 73, 'result': 30, 'trigger': 27, 'come': 26, 'produce': 21, 'generate': 18, 'lead': 15, 'get': 11, 'make': 11, 'radiate': 11, 'reduce': 9, 'emit': 8, 'have': 6, 'instigate': 6, 'i': 5, 'arise': 5, 'catch': 5, 'start': 5, 'experience': 5, 'prevent': 5, 'stem': 5, 'create': 5, 'establish': 4, 'Cause': 4, 'give': 4, 'derive': 4, 'sell': 4, 'treat': 4, 'repair': 4, 'suffer': 4, 'turn': 3, 'contribute': 3, 'originate': 3, 'relieve': 3, 'ensue': 3, 'consider': 3, 'identify': 3, 'develop': 3, 'hit': 3, 'face': 3, 'reveal': 3, 'feel': 3, 'contain': 2, 'affect': 2, 'regard': 2, 'responsible': 2, 'spill': 2, 'scream': 2, 'resolve': 2, 'destroy': 2, 'describe': 2, 'alleviate': 2, 'gain': 2, 'root': 2, 'put': 2, 'set': 2, 'show': 2, 'help': 2, 'increase': 2, 'withdraw': 2, 'fear': 2, 'recognize': 2, 'follow': 2, 'br': 2, 'reach': 2, 'provide': 2, 'minimize': 2, 'find': 2, 'threaten': 2, 'trigger.': 1, 'sw': 1, 'close': 1, 'preclude': 1, 'release': 1, 'mark': 1, 'cau': 1, 'offer': 1, 'enter': 1, 'convert': 1, 'acknowledge': 1, 'push': 1, 'avoid': 1, 'transmit': 1, 'cite': 1, 'source': 1, 'strike': 1, 'take': 1, 'burst': 1, 'achieve': 1, 'correct': 1, 'account': 1, 'inspect': 1, 'entitle': 1, 'relate': 1, 'output': 1, 'attribut': 1, 'ensure': 1, 'move': 1, 'play': 1, 'become': 1, 'fall': 1, 'combat': 1, 'suffocation': 1, 'cause\n': 1, 'stimulate': 1, 'transact': 1, 'visit': 1, 'depend': 1, 'reverse': 1, 'run': 1, 'direct': 1, 'make.': 1, 'tug': 1, 'detach': 1, 'eat': 1, 'conceptualize': 1, 'neutralize': 1, 'arrive': 1, 'proclaim': 1, 'initiate': 1, 'read': 1, 'survive': 1, 'shape': 1, 'asses': 1, 'unite': 1, 'prepare': 1, 'know': 1, 'translate': 1, 'craft': 1, 'deprive': 1, 'retain': 1, 'mov': 1, 'decide': 1, 'support': 1, 'eliminate': 1, 'depart': 1, 'cough': 1, 'boost': 1, 'grow': 1, 'weaken': 1, 'reside': 1, 'consist': 1, 'batter': 1, 'send': 1, 'expose': 1, 'succumb': 1, 'share': 1, 'burn': 1, 'overstate': 1, 'accommodate': 1, 'contract': 1, 'check': 1, 'inundate': 1, 'donate': 1, 'end': 1, 'Place': 1, 'offset': 1, 'spread': 1, 'fill': 1, 'subject': 1, 'intubation': 1, 'receive': 1, 'essential': 1, 'mitigate': 1, 'choke': 1, 'await': 1, 'quench': 1, 'change': 1, 'improve': 1, 'see': 1, 'teach': 1, 'enhance': 1, 'sale_of': 1, 'resist': 1, 'drink': 1, 'smile': 1, 'demonstrate': 1, 'happen': 1, 'occur': 1, 'bombard': 1, 'include': 1, 'allow': 1, 'delay': 1, 'sleep': 1, 'respond': 1, 'block': 1, 'brace': 1, 'producean': 1, 'after': 1, 'obtain': 1, 'examine': 1, 'mediate': 1, 'emanate': 1, 'consumption': 1, 'beget': 1, 'utter': 1, 'connect': 1, 'counteract': 1}

MC = {'be': 77, 'have': 27, 'find': 11, 'consist': 11, 'join': 8, 'bring': 8, 'come': 8, 'contain': 7, 'become': 6, 'make': 6, 'take': 6, 'form': 6, 'establish': 6, 'attack': 5, 'build': 5, 'stand': 5, 'live': 5, 'see': 5, 'develop': 5, 'comprise': 4, 'drive': 4, 'gather': 4, 'compose': 4, 'start': 4, 'produce': 4, 'spot': 4, 'send': 4, 'create': 4, 'fly': 4, 'include': 3, 'call': 3, 'serve': 3, 'keep': 3, 'get': 3, 'study': 3, 'leave': 3, 'arrive': 3, 'fall': 3, 'select': 3, 'organize': 3, 'maintain': 3, 'watch': 3, 'lead': 3, 'provide': 3, 'show': 3, 'perform': 3, 'support': 3, 'work': 3, 'collect': 2, 'pull': 2, 'move': 2, 'pas': 2, 'swim': 2, 'observe': 2, 'reflect': 2, 'investigate': 2, 'increase': 2, 'migrate': 2, 'splash': 2, 'prove': 2, 'dwell': 2, 'recapture': 2, 'employ': 2, 'turn': 2, 'use': 2, 'play': 2, 'pick': 2, 'purchase': 2, 'carry': 2, 'return': 2, 'meet': 2, 'describe': 2, 'sign': 2, 'expose': 2, 'sell': 2, 'i': 2, 'represent': 2, 'fee': 2, 'involve': 2, 'issue': 2, 'set': 2, 'follow': 2, 'harbor': 2, 'announce': 2, 'descend': 2, 'face': 2, 'train': 2, 'associate': 2, 'decide': 2, 'mobilize': 2, 'fight': 2, 'constitute': 2, 'witnes': 2, 'hire': 2, 'surround': 2, 'drift': 2, 'handle': 2, 'shake': 2, 'enter': 2, 'look': 2, 'back': 1, 'amount': 1, 'lie': 1, 'degenerate': 1, 'arm': 1, 'complement': 1, 'challenge': 1, 'Pas': 1, 'offer': 1, 'direct': 1, 'bear': 1, 'visit': 1, 'lure': 1, 'tremble': 1, 'exposure': 1, 'raise': 1, 'organise': 1, 'board': 1, 'attend': 1, 'perch': 1, 'hear': 1, 'marvell': 1, 'impres': 1, 'assemble': 1, 'lay': 1, 'continue': 1, 'plague': 1, 'enroll': 1, 'cause': 1, 'deploy': 1, 'cros': 1, 'pay': 1, 'manage': 1, 'encounter': 1, 'skulk': 1, 'identify': 1, 'member': 1, 'feel': 1, 'storm': 1, 'abandon': 1, 'devise': 1, 'wait': 1, 'serenade': 1, 'dash': 1, 'secure': 1, 'fill': 1, 'cl': 1, 'take.': 1, 'float': 1, 'bask': 1, 'choose': 1, 'plow': 1, 'experience': 1, 'chair': 1, 'stack': 1, 'arrest': 1, 'receive': 1, 'reconstitute': 1, 'depict': 1, 'connect': 1, 'initiate': 1, 'say': 1, 'draw': 1, 'coo': 1, 'focu': 1, 'reduce': 1, 'presid': 1, 'loan': 1, 'scavenge': 1, 'run': 1, 'bellow': 1, 'attract': 1, 'embrace': 1, 'feast': 1, 'glid': 1, 'Interact': 1, 'burst': 1, 'Overrun': 1, 'resign': 1, 'vote': 1, 'drop': 1, 'exist': 1, 'trot': 1, 'sire': 1, 'entertain': 1, 'seize': 1, 'share': 1, 'suspend': 1, 'change': 1, 'examine': 1, 'forge': 1, 'close': 1, 'appear': 1, 'contend': 1, 'infest': 1, 'reteam': 1, 'struggle': 1, 'defeat': 1, 'conquer': 1, 'invest': 1, 'rent': 1, 'cadge': 1, 'hang': 1, 'acquir': 1, 'obtain': 1, 'assert': 1, 'point': 1, 'hunt': 1, 'rope': 1, 'integrate': 1, 'ease': 1, 'place': 1, 'discus': 1, 'emerge': 1, 'rail': 1, 'remind': 1, 'fit': 1, 'teach': 1, 'depend': 1, 'replace': 1, 'bind': 1, 'notice': 1, 'deny': 1, 'cover': 1, 'jostl': 1, 'charm': 1, 'arrange': 1, 'skip': 1, 'comment': 1, 'crouch': 1, 'sound': 1, 'abduct': 1, 'aim': 1, 'Br': 1, 'accompany': 1, 'assault': 1, 'cast': 1, 'destroy': 1, 'ingest': 1, 'begin': 1, 'rely': 1, 'stare': 1, 'grant': 1, 'speak': 1, 'appoint': 1, 'party': 1, 'portray': 1, 'stay': 1, 'devote': 1, 'order': 1, 'soar': 1, 'purr': 1, 'auction': 1, 'expect': 1, 'sacrifice': 1, 'result': 1, 'battle': 1, 'belong': 1, 'advocate': 1, 'contribute': 1, 'pit': 1, 'betray': 1, 'catch': 1, 'recall': 1, 'tumble': 1, 'tease': 1, 'yell': 1, 'tell': 1, 'march': 1, 'promote': 1, 'contract': 1, 'feature': 1, 'step': 1, 'brave': 1, 'resemble': 1, 'reject': 1, 'nest': 1, 'shift': 1, 'deal': 1, 'stumble': 1, 'imagine': 1, 'name': 1, 'scurry': 1, 'convene': 1, 'generate': 1, 'motivat': 1, 'adapt': 1, 'try': 1, 'undulate': 1, 'conceptualize': 1, 'discover': 1, 'swarm': 1, 'read': 1, 'season': 1, 'appeal': 1, 'realize': 1, 'drown': 1, 'unfold': 1, 'populate': 1, 'trigger': 1, 'rescue': 1, 'sight': 1, 'mark': 1, 'supply': 1, 'confirm': 1, 'walk': 1, 'undertake': 1, 'detect': 1, 'survive': 1, 'latch': 1, 'traverse': 1, 'convert': 1, 'flee': 1, 'satisfy': 1, 'gallop': 1, 'erupt': 1, 'utilization': 1, 'advance': 1, 'locate': 1, 'sail': 1, 'Picture': 1, 'exhume': 1, 'publish': 1, 'approve': 1, 'compete': 1, 'predicate': 1, 'evaluate': 1, 'trim': 1, 'let': 1, 'launch': 1, 'break': 1, 'scold': 1, 'indicate': 1, 'explain': 1, 'whirl': 1, 'fortify': 1, 'bounce': 1, 'gamble': 1, 'chase': 1, 'prepare': 1, 'evince': 1, 'cal': 1, 'breathe': 1}

MT = {'be': 48, 'describe': 20, 'make': 18, 'discus': 16, 'reflect': 15, 'explain': 13, 'relate': 12, 'have': 10, 'depict': 10, 'contain': 10, 'report': 10, 'give': 9, 'provide': 9, 'explore': 9, 'investigate': 8, 'point': 8, 'examine': 8, 'illustrate': 7, 'define': 7, 'inform': 7, 'document': 7, 'advertise': 6, 'center': 6, 'declare': 6, 'concern': 6, 'outline': 6, 'focu': 6, 'become': 6, 'deal': 6, 'talk': 5, 'form': 5, 'publish': 5, 'supply': 5, 'cover': 5, 'summarise': 5, 'consider': 5, 'set': 5, 'narrate': 4, 'offer': 4, 'devote': 4, 'apply': 4, 'show': 4, 'study': 4, 'criticize': 4, 'run': 4, 'tell': 4, 'speak': 4, 'feature': 4, 'develop': 4, 'comment': 4, 'detail': 4, 'receive': 4, 'introduce': 4, 'come': 4, 'tackle': 4, 'highlight': 4, 'appear': 4, 'mention': 3, 'analyse': 3, 'prepare': 3, 'include': 3, 'find': 3, 'release': 3, 'assert': 3, 'swirl': 3, 'learn': 3, 'create': 3, 'adopt': 3, 'addres': 3, 'involve': 3, 'compare': 3, 'reveal': 3, 'survey': 3, 'state': 3, 'remind': 2, 'convene': 2, 'analyze': 2, 'obtain': 2, 'promote': 2, 'raise': 2, 'produce': 2, 'hold': 2, 'portray': 2, 'use': 2, 'teach': 2, 'send': 2, 'insist': 2, 'criticise': 2, 'take': 2, 'go': 2, 'review': 2, 'base': 2, 'chart': 2, 'conduct': 2, 'present': 2, 'pertain': 2, 'encourage': 2, 'establish': 2, 'govern': 2, 'accept': 2, 'expres': 2, 'write': 2, 'allege': 2, 'stimulate': 2, 'br': 2, 'conclude': 2, 'issue': 1, 'construct': 1, 'incorporate': 1, 'launch': 1, 'respond': 1, 'Be': 1, 'inquire': 1, 'comprise': 1, 'represent': 1, 'emphasise': 1, 'serve': 1, 'rotate': 1, 'achieve': 1, 'differ': 1, 'provoke': 1, 'submit': 1, 'engage': 1, 'begin': 1, 'indicate': 1, 'witnes': 1, 'pas': 1, 'recount': 1, 'attribute': 1, 'summarize': 1, 'furnish': 1, 'dominate': 1, 'surround': 1, 'enter': 1, 'keep': 1, 'solve': 1, 'approache': 1, 'discusse': 1, 'trace': 1, 'approach': 1, 'direct': 1, 'suspend': 1, 'retrieve': 1, 'drive': 1, 'consider.': 1, 'reach': 1, 'emphasize': 1, 'add': 1, 'evolve': 1, 'erupt': 1, 'meet': 1, 'ad': 1, 'paint': 1, 'capture': 1, 'facilitate': 1, 'say': 1, 'occur': 1, 'pronounce': 1, 'demand': 1, 'design.': 1, 'need': 1, 'tabulate': 1, 'propose': 1, 'specialize': 1, 'answer': 1, 'move': 1, 'get': 1, 'sw': 1, 'refer': 1, 'open': 1, 'limit': 1, 'look': 1, 'make.': 1, 'dedicate': 1, 'explore.': 1, 'organise': 1, 'regulate': 1, 'piece': 1, 'position': 1, 'connect': 1, 'characterize': 1, 'increase': 1} 

ED = {'place': 56, 'put': 53, 'move': 44, 'pour': 39, 'send': 34, 'migrate': 27, 'release': 27, 'add': 21, 'spread': 20, 'deliver': 19, 'insert': 19, 'arrive': 19, 'drop': 18, 'drain': 16, 'import': 15, 'throw': 15, 'blow': 14, 'invest': 14, 'travel': 13, 'donate': 13, 'transport': 12, 'take': 12, 'stuff': 11, 'inject': 11, 'pas': 11, 'export': 10, 'leak': 10, 'enter': 10, 'carry': 9, 'push': 9, 'run': 9, 'flow': 9, 'br': 9, 'go': 8, 'dump': 8, 'drag': 8, 'land': 8, 'journey': 8, 'implant': 8, 'fetch': 7, 'give': 7, 'ship': 7, 'hand': 7, 'fly': 6, 'remove': 6, 'fall': 6, 'reschedule': 6, 'misplace': 6, 'sink': 5, 'approach': 5, 'postpone': 4, 'pack': 4, 'award': 4, 'be': 4, 'enclose': 4, 'seal': 3, 'post': 3, 'store': 3, 'Pour': 3, 'bestow': 3, 'lock': 2, 'start': 2, 'come': 2, 'collect': 2, 'install': 2, 'load': 1, 'incorporate': 1, 'slip': 1, 'inherit': 1, 'transfer': 1, 'deposit': 1, 'Move': 1, 'sail': 1, 'retract': 1, 'stir': 1, 'mix': 1, 'locate': 1, 'boost': 1, 'Add': 1, 'hide': 1, 'arriv': 1, 'rescue': 1, 'carve': 1, 'remain': 1, 'spool': 1, 'fill': 1, 'look': 1, 'overcharge': 1, 'supply': 1, 'Drain': 1, 'crash': 1, 'bring': 1, 'beat': 1, 'meet': 1, 'hold': 1, 'show': 1, 'bank': 1, 'gather': 1, 'filter': 1, 'Take': 1, 'relieve': 1, 'concern': 1, 'change': 1, 'include': 1, 'bury': 1, 'spend': 1, 'transmit': 1, 'grant': 1, 'leak.': 1, 'extractant': 1, 'integrate': 1, 'introduce': 1, 'stash': 1, 'make': 1, 'identify': 1}

CW = {'be': 94, 'have': 55, 'contain': 33, 'comprise': 30, 'include': 26, 'make': 22, 'hold': 12, 'consist': 12, 'compose': 12, 'use': 11, 'show': 11, 'attach': 10, 'place': 8, 'look': 7, 'connect': 7, 'provide': 7, 'move': 7, 'find': 7, 'develop': 6, 'open': 6, 'work': 6, 'appear': 6, 'br': 6, 'i': 6, 'feature': 6, 'form': 6, 'start': 6, 'take': 6, 'break': 5, 'click': 5, 'rotate': 5, 'set': 5, 'pres': 5, 'locate': 5, 'extend': 4, 'design': 4, 'construct': 4, 'carve': 4, 'display': 4, 'combine': 4, 'remove': 4, 'fit': 4, 'store': 4, 'present': 4, 'create': 4, 'keep': 4, 'mount': 4, 'fill': 4, 'see': 4, 'enclose': 4, 'divide': 3, 'result': 3, 'stand': 3, 'require': 3, 'rest': 3, 'decorate': 3, 'depend': 3, 'grow': 3, 'publish': 3, 'increase': 3, 'clean': 3, 'allow': 3, 'hang': 3, 'reflect': 3, 'accommodate': 3, 'run': 3, 'install': 3, 'measure': 3, 'push': 3, 'build': 3, 'perform': 3, 'involve': 3, 'reduce': 2, 'report': 2, 'reach': 2, 'regulate': 2, 'insert': 2, 'notice': 2, 'anchor': 2, 'lock': 2, 'wrap': 2, 'paint': 2, 'fix': 2, 'carry': 2, 'stretch': 2, 'come': 2, 'receive': 2, 'relate': 2, 'hit': 2, 'wear': 2, 'grab': 2, 'detach': 2, 'pull': 2, 'lift': 2, 'drag': 2, 'summarise': 2, 'touch': 2, 'support': 2, 'proces': 2, 'force': 2, 'draw': 2, 'feel': 2, 'lie': 2, 'operate': 2, 'transmit': 2, 'catch': 2, 'check': 2, 'walk': 2, 'launch': 2, 'compris': 2, 'judge': 2, 'pas': 2, 'correspond': 2, 'release': 2, 'give': 2, 'cause': 2, 'follow': 2, 'fall': 2, 'mark': 2, 'put': 2, 'power': 2, 'go': 2, 'eliminate': 1, 'pad': 1, 'stop': 1, 'leave': 1, 'Unwind': 1, 'fly': 1, 'span': 1, 'permit': 1, 'collapse': 1, 'recur': 1, 'get': 1, 'filter': 1, 'riffle': 1, 'control': 1, 'list': 1, 'zero': 1, 'engage': 1, 'account': 1, 'arrange': 1, 'unscrew': 1, 'shape': 1, 'probe': 1, 'choose': 1, 'prim': 1, 'shorten': 1, 'couple': 1, 'offer': 1, 'undertake': 1, 'snorgle': 1, 'flare': 1, 'lead': 1, 'unlock': 1, 'hatchet': 1, 'date': 1, 'scale': 1, 'curl': 1, 'strike': 1, 'cultivate': 1, 'dedicate': 1, 'survive': 1, 'load': 1, 'raise': 1, 'throw': 1, 'function': 1, 'play': 1, 'consider': 1, 'disregard': 1, 'vibrate': 1, 'expenditure': 1, 'guide': 1, 'signal': 1, 'cut': 1, 'Embroidered.': 1, 'pierc': 1, 'settle': 1, 'fee': 1, 'Recognized': 1, 'Point': 1, 'maximize': 1, 'pry': 1, 'circulate': 1, 'rig': 1, 'twist': 1, 'modify': 1, 'depict': 1, 'drive': 1, 'expos': 1, 'absorb': 1, 'shear': 1, 'hide': 1, 'collect': 1, 'roam': 1, 'scrape': 1, 'threaten': 1, 'position': 1, 'fire': 1, 'complement': 1, 'slice': 1, 'clasp': 1, 'become': 1, 'coupl': 1, 'observe': 1, 'stab': 1, 'pop': 1, 'stun': 1, 'protect': 1, 'bend': 1, 'curtain': 1, 'record': 1, 'grip': 1, 'redesign': 1, 'narrate': 1, 'help': 1, 'convince': 1, 'spin': 1, 'gleam': 1, 'malfunction': 1, 'swim': 1, 'lubricate': 1, 'change': 1, 'adjust': 1, 'slip': 1, 'root': 1, 'addres': 1, 'search': 1, 'remember': 1, 'scatter': 1, 'manufacture': 1, 'occur': 1, 'cleat': 1, 'thrust': 1, 'adopt': 1, 'grasp': 1, 'cover': 1, 'swish': 1, 'foresee': 1, 'calculate': 1, 'lower': 1, 'perfect': 1, 'refer': 1, 'begin': 1, 'heal': 1, 'paddle': 1, 'retract': 1, 'preside': 1, 'reveal': 1, 'post': 1, 'mean': 1, 'stick': 1, 'study': 1, 'graz': 1, 'kis': 1, 'opt': 1, 'cock': 1, 'beat': 1, 'contribute': 1, 'align': 1, 'flow': 1, 'flood': 1, 'purchase': 1, 'Assist': 1, 'insulate': 1, 'sense': 1, 'heat': 1, 'expand': 1, 'convert': 1, 'base': 1, 'float': 1, 'mix': 1, 'stay': 1, 'meet': 1, 'capture': 1, 'maintain': 1, 'save': 1, 'prepare': 1, 'withstand': 1, 'complain': 1, 'situate': 1, 'lose': 1, 'close': 1, 'improve': 1, 'wax': 1, 'eat': 1, 'detect': 1, 'clip': 1, 'belong': 1, 'fasten': 1, 'predict': 1, 'weaken': 1, 'scrub': 1, 'vary': 1, 'inspect': 1, 'publish.': 1, 'ascend': 1, 'scratch': 1, 'plant': 1, 'overhang': 1, 'average': 1, 'acces': 1, 'dangle': 1, 'sw': 1, 'lean': 1, 'tune': 1, 'thicken': 1, 'locate.': 1, 'prevent': 1, 'solder': 1, 'produce': 1, 'ha': 1, 'empty': 1, 'appreciate': 1, 'love': 1, 'procure': 1, 'hatch': 1, 'secrete': 1, 'block': 1, 'read': 1, 'behave': 1, 'explore': 1, 'erode': 1, 'gain': 1, 'recognize': 1, 'digest': 1, 'track': 1, 'collaborate': 1, 'analyze': 1, 'seek': 1, 'reinforce': 1, 'rise': 1, 'blink': 1, 'Combine': 1, 'step': 1, 'bounce': 1, 'crush': 1, 'delete': 1, 'outfit': 1, 'enter': 1, 'add': 1, 'consume': 1, 'share': 1, 'assist': 1, 'drape': 1}

CC = {'find': 24, 'store': 22, 'enclose': 15, 'discover': 14, 'hide': 12, 'keep': 12, 'be': 11, 'lock': 10, 'hold': 10, 'contain': 8, 'include': 8, 'carry': 7, 'wa': 7, 'put': 6, 'have': 5, 'seal': 4, 'give': 4, 'make': 4, 'br': 3, 'i': 3, 'place': 3, 'send': 3, 'sit': 3, 'contained.': 3, 'show': 3, 'full': 3, 'see': 3, 'eat': 2, 'strike': 2, 'serve': 2, 'carried.': 2, 'get': 2, 'take': 2, 'fit': 2, 'in': 2, 'pick': 2, 'cover': 2, 'open': 2, 'come': 2, 'tuck': 2, 'use': 2, 'cook': 2, 'bring': 2, 'goe': 1, 'separate': 1, 'preserve': 1, 'coil': 1, 'stor': 1, 'scratch': 1, 'secure': 1, 'arrive': 1, 'fold': 1, 'intend': 1, 'introduce': 1, 'deliver': 1, 'hang': 1, 'receive': 1, 'throw': 1, 'insert': 1, 'weigh': 1, 'prefer': 1, 'kill': 1, 'bear': 1, 'label': 1, 'attach': 1, 'return': 1, 'inside': 1, 'connect': 1, 'Inside': 1, 'found.': 1, 'mis': 1, 'cost': 1, 'folder': 1, 'print': 1, 'pull': 1, 'unpack': 1, 'fill': 1, 'stand': 1, 'search': 1, 'plac': 1, 'accept': 1, 'sup': 1, 'fix': 1, 'traverse': 1, 'filled\twith': 1, 'stir': 1, 'comprise': 1, 'measure': 1, 'dissolve': 1, 'inventoried': 1, 'press': 1, 'house': 1, 'Mak': 1, 'examine': 1, 'leave': 1, 'tak': 1, 'measur': 1, 'explode': 1, 'talk': 1, 'pas': 1, 'become': 1, 'smuggle': 1, 'sleep': 1, 'look': 1, 'snatch': 1, 'arrange': 1, 'remove': 1, 'offer': 1, 'cram': 1, 'mark': 1, 'face': 1, 'ship': 1, 'try': 1, 'impart': 1, 'alert': 1, 'stored.': 1, 'spill': 1, 'protect': 1, 'conceal': 1, 'saw': 1, 'fly': 1, 'deposit': 1, 'play': 1, 'escape': 1, 'lay.': 1, 'buy': 1, 'display': 1, 'market': 1, 'enter': 1, 'seize': 1, 'check': 1, 'grow': 1, 'mistake': 1, 'notice': 1}

EO = {'be': 86, 'contain': 65, 'find': 29, 'store': 23, 'enclose': 19, 'discover': 14, 'lock': 12, 'keep': 11, 'hide': 10, 'place': 9, 'carry': 9, 'put': 9, 'hold': 9, 'wa': 8, 'have': 6, 'br': 5, 'include': 5, 'seal': 4, 'fill': 4, 'make': 4, 'serve': 3, 'give': 3, 'sit': 3, 'cover': 3, 'open': 3, 'see': 3, 'eat': 2, 'i': 2, 'wrap': 2, 'send': 2, 'fit': 2, 'measure': 2, 'pull': 2, 'tuck': 2, 'use': 2, 'determine': 2, 'hav': 2, 'container': 2, 'explode': 2, '"contains"': 2, 'leave': 2, 'remove': 2, 'Wa': 2, 'go': 1, 'date': 1, 'preserve': 1, 'lose': 1, 'strike': 1, 'coil': 1, 'stor': 1, 'scratch': 1, 'secure': 1, 'arrive': 1, 'attach': 1, 'introduce': 1, 'symbolise': 1, 'hang': 1, 'receive': 1, 'throw': 1, 'weigh': 1, 'kill': 1, 'label': 1, 'fall': 1, 'pour': 1, 'note': 1, 'connect': 1, 'cost': 1, '"shows"': 1, 'print': 1, 'unpack': 1, 'stand': 1, 'take': 1, 'full': 1, 'own': 1, 'accept': 1, 'rattle': 1, 'drink': 1, 'ensure': 1, 'cease': 1, 'ha': 1, 'stir': 1, 'ensconce': 1, 'dissolve': 1, 'generate': 1, 'inventory': 1, 'house': 1, 'examine': 1, 'tak': 1, 'inside': 1, 'Find': 1, '"inside"': 1, 'bury': 1, 'Remember': 1, 'show': 1, 'pas': 1, 'auction.': 1, 'turn': 1, 'become': 1, 'See': 1, 'sleep': 1, 'stuff': 1, 'snatch': 1, 'arrange': 1, 'offer': 1, 'cram': 1, 'retriev': 1, 'ship': 1, 'try': 1, 'impart': 1, 'bind': 1, 'protect': 1, 'conceal': 1, 'fly': 1, 'get': 1, 'deposit': 1, 'lock.': 1, 'play': 1, 'escape': 1, 'display': 1, 'come': 1, 'enter': 1, 'sort': 1, 'seize': 1, 'check': 1, 'grow': 1, 'mistake': 1, 'Notice': 1, 'surround': 1}

PP = {'be': 103, 'come': 35, 'make': 25, 'derive': 22, 'leave': 21, 'arrive': 20, 'originate': 20, 'run': 15, 'have': 13, 'take': 12, 'distill': 12, 'farm': 11, 'produce': 10, 'release': 10, 'go': 10, 'descend': 9, 'fall': 9, 'extract': 8, 'manufacture': 8, 'start': 7, 'obtain': 7, 'emerge': 7, 'remove': 6, 'pop': 6, 'use': 5, 'get': 5, 'root': 4, 'move': 4, 'leak': 4, 'abscond': 4, 'hail': 3, 'evolve': 3, 'develop': 3, 'add': 3, 'evaporate': 3, 'elude': 3, 'send': 3, 'br': 3, 'form': 3, 'cook': 3, 'involve': 3, 'include': 3, 'put': 3, 'generate': 3, 'sail': 3, 'drink': 3, 'gain': 2, 'arise': 2, 'flourish': 2, 'grow': 2, 'become': 2, 'collect': 2, 'look': 2, 'found.': 2, 'break': 2, 'taxi': 2, 'date': 2, 'try': 2, 'pour': 2, 'taste': 2, 'refer': 2, 'describe': 2, 'travel': 2, 'feature': 2, 'bail': 2, 'construct': 2, 'emanate': 2, 'gather': 2, 'see': 2, 'give': 2, 'treat': 2, 'receive': 2, 'alarm': 1, 'distil': 1, 'yield': 1, 'drive': 1, 'bloom': 1, 'escape': 1, 'act': 1, 'delve': 1, 'tear': 1, 'identify': 1, 'deliver': 1, 'sequence': 1, 'reach': 1, 'fire': 1, 'fund': 1, 'depart': 1, 'create': 1, 'steal.': 1, 'radiate': 1, 'achieve': 1, 'end': 1, 'win': 1, 'represent': 1, 'consider': 1, 'infiltrate': 1, 'organize': 1, 'follow': 1, 'deice': 1, 'attribute': 1, 'juxtapose': 1, 'strengthen': 1, 'flee': 1, 'prove': 1, 'combine': 1, 'emerge.': 1, 'recall': 1, 'exit': 1, 'twist': 1, 'update': 1, 'graduate': 1, 'convert': 1, 'save': 1, 'mean': 1, 'set': 1, 'return': 1, 'offer': 1, 'display': 1, 'supervise': 1, 'draw': 1, 'estimate': 1, 'review': 1, 'serve': 1, 'respond': 1, 'explain': 1, 'learn': 1, 'reveal': 1, 'consult': 1, 'order': 1, 'pres': 1, 'play': 1, 'climb': 1, 'base': 1, 'direct': 1, 'compose': 1, 'push': 1, 'be_given': 1, 'understand': 1, 'pull': 1, 'resign': 1, 'incorporate': 1, 'eject': 1, 'submit': 1, 'stimulate': 1, 'copy': 1, 'be_surprised': 1, 'solicit': 1, 'provide': 1, 'watch': 1, 'switch': 1, 'evacuate': 1, 'find.': 1, 'i': 1, 'depend': 1, 'qualify': 1, 'say': 1, 'investigate': 1, 'sample': 1, 'catch': 1, 'throw': 1, 'group': 1, 'cure': 1, 'fuel': 1, 'popularize': 1, 'begin': 1, 'bake': 1, 'step': 1, 'notify': 1, 'star': 1, 'drown.': 1, 'mention': 1, 'reschedule': 1, 'hand': 1, 'uprise': 1, 'lie': 1, 'kick': 1, 'leap': 1, 'sweeten': 1, 'stem': 1, 'secure': 1, 'remain': 1, 'demolish.': 1, 'replicate': 1, 'find': 1, 'better': 1, 'lodge': 1, 'retrieve': 1, 'share': 1, 'clog': 1, 'attract': 1, 'perform': 1, 'work': 1, 'meet': 1, 'report': 1, 'steal': 1, 'harm': 1, 'recruit': 1, 'compile': 1, 'outline': 1, 'cause': 1, 'prepare': 1, 'Reject': 1}

DICTS = [(IA, 'IA'), (PP, 'PP'), (CE, 'CE'), (MC, 'MC'),
         (EO, 'EO'), (MT, 'MT'), (ED, 'ED'), (CW, 'CW'), (CC, 'CC')]


LOG_DICTS = []
for d in DICTS:
    LOG_DICTS.append(({key: math.log10(value)
                     for key, value in d[0].items()}, d[1]))

def softmax(x):
    # Compute softmax values for each sets of scores in x.
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


# verb stemmen
def get_verb_lemma(verb: str) -> str:
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(verb, pos='v')

# mit dicts vergleichen
def predict_class(bert_predictions, class_dicts=LOG_DICTS):
    scores = []
    score = 0
    for d in class_dicts:
        for prediction in bert_predictions:
            try:
                verb = get_verb_lemma(prediction['token_str'])
                score += d[0][verb] * prediction['score']
            except KeyError:
                continue

        scores.append((score))
        score = 0
    
    # softmax and sort scores and add labels
    softmaxed_scores = softmax(np.array(scores))
    return softmaxed_scores

In [17]:
# optional: get tagged words marked in output by providing original sentence with XML tags
orig_sentence = "Their <e1>composer</e1> has sunk into <e2>oblivion</e2>."
xml_indices = []
for index, token in enumerate(orig_sentence.split(' ')):
    if re.search('<e[12]>.*</e[12]>', token):
        xml_indices.append(index)

<e1>composer</e1>
<e2>oblivion</e2>.


In [9]:
sentence = "Their composer has [MASK] into oblivion."

# first get the prediction from the original sentence so we have something to compare our next results with
bert_pred = bert_predict(sentence)
class_pred = predict_class(bert_pred)

class_predictions = []
for sub_sentence in get_sub_sentences(sentence):
    bert_sub_pred = bert_predict(sub_sentence)
    class_sub_pred = predict_class(bert_sub_pred)
    class_predictions.append(class_sub_pred)

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
All PyTorch model weights were used when initializing TFBertForM

In [5]:
cosine_similarities = []
for p in class_predictions:
    cosine_similarities.append(get_cosine_sim(p, class_pred))


# Calculate min and max
X_min = min(cosine_similarities)
X_max = max(cosine_similarities)

# Scale the array
scaled_similarities = [(x - X_min) / (X_max - X_min) for x in cosine_similarities]

In [6]:
scaled_similarities

[0.997315840000648,
 0.051165829309736066,
 0.6585034365157626,
 0.8015529225774787,
 1.0,
 0.0,
 0.5588536877684778,
 0.9130150763118824,
 0.49670561444338185]

In [8]:
from IPython.display import HTML

STR = "<div style='display: flex; flex-direction: row; font-size: xx-large; '>"
for index, token in enumerate(sentence[:-1].split(' ')):
    STR += f"<div style='color: rgb(255, {scaled_similarities[index] * 255}, {scaled_similarities[index] * 255})'>{token}&nbsp</div>"
STR = STR[:-11] + "</div>.</div>"

HTML(STR)

In [174]:
html_str = """
<script>
function js_in_ipynb_lul(){
    alert("brrrrrrrrrrt fuer die welt!")
}
</script>
<button onclick="js_in_ipynb_lul()">click me</button>
"""

HTML(html_str)